In [ ]:
"""Recursion. Decorator. Generator."""

**кеширование с помощью lru_cache**

### 🧠 Объяснение `lru_cache` на пальцах  

`lru_cache` — это декоратор, который **кеширует** (запоминает) результаты вызовов функции, чтобы не вычислять их заново.  

#### 🔹 Как это работает в `fib(n)`?  
1. **Первый вызов `fib(35)`**  
   - Функция начинает рекурсивно считать `fib(34) + fib(33)`, потом `fib(33) + fib(32)` и так далее.  
   - **Каждый новый результат записывается в кеш.**  

2. **Последующие вызовы `fib(35)`**  
   - Если функция вызывается с тем же аргументом (`n=35`), она **не вычисляется заново**, а берётся из кеша.  
   - Поэтому время выполнения сокращается с секунд до микросекунд.  

---

### 📂 Как хранятся значения?  
`lru_cache` использует **словарь** (как `dict`), где:  
- **Ключ** → аргументы функции (например, `n=35`).  
- **Значение** → результат (`fib(35)`).  

#### 🔹 `maxsize=1000`  
- Это ограничение на количество запоминаемых результатов.  
- Если вызвать `fib(1001)`, то самый старый результат (`fib(1)`) удалится (LRU = Least Recently Used).  

#### 🔹 Пример кеша после вычисления `fib(5)`  
```python
{
    0: 1,  # fib(0)
    1: 1,  # fib(1)
    2: 2,  # fib(2) = fib(1) + fib(0)
    3: 3,  # fib(3) = fib(2) + fib(1)
    4: 5,  # fib(4) = fib(3) + fib(2)
    5: 8,  # fib(5) = fib(4) + fib(3)
}
```
Если теперь вызвать `fib(4)`, значение `5` возьмётся из кеша, а не будет вычисляться заново.  

---

### ⚡ Почему `fib(35)` работает мгновенно после первого вызова?  
Потому что все промежуточные результаты (`fib(0)`, `fib(1)`, ..., `fib(35)`) уже сохранены в кеше!  

Без `lru_cache` время было бы **очень долгим**, потому что функция считала бы одни и те же значения миллионы раз (например, `fib(2)` вызывается в рекурсии тысячи раз).  

---

### 💡 Вывод  
`lru_cache` — это **"волшебная память"** для функций, которые часто вызываются с одинаковыми аргументами.  
- Ускоряет код в **сотни раз** (если много повторных вычислений).  
- Тратит **немного памяти** на хранение результатов.  

Если убрать `@lru_cache`, время выполнения `fib(35)` будет **секунды**, а не микросекунды! 🚀

### 🔍 **Как работает `lru_cache` и его методы .cache_clear() и .cache_info()**

#### **1. Основной принцип `lru_cache`**
`lru_cache` — это декоратор, который **запоминает результаты вызовов функции**, чтобы избежать повторных вычислений.  

- **Кеш хранится в виде словаря**:  
  - **Ключ** → аргументы функции (например, `n=5` для `fib(5)`).  
  - **Значение** → результат вычисления (например, `8` для `fib(5)`).  
- **LRU = Least Recently Used** → если кеш переполнен, удаляются **самые старые** (реже всего используемые) значения.  

#### **2. Методы `lru_cache`**
У декорированной функции появляются два полезных метода:  

1. **`.cache_info()` → статистика использования кеша**  
   Показывает:  
   - `hits` — сколько раз результат взят из кеша (ускорение).  
   - `misses` — сколько раз пришлось вычислять.  
   - `maxsize` — максимальный размер кеша.  
   - `currsize` — текущее количество закешированных значений.  

2. **`.cache_clear()` → полная очистка кеша**  
   Удаляет все сохранённые результаты. Полезно, если:  
   - данные устарели,  
   - нужно освободить память,  
   - вы тестируете производительность.  

---

### 🔧 **Пример 1: Как работает кеширование**
Рассмотрим функцию `square()`, которая вычисляет квадрат числа:

```python
from functools import lru_cache


@lru_cache(maxsize=3)  # храним только 3 последних результата
def square(x):
    print(f"Вычисляю квадрат {x}...")  # Печатаем только при реальном вычислении
    return x * x


# Первые вызовы — "misses" (кеш пуст)
print(square(2))  # Вычисляю квадрат 2... → 4
print(square(3))  # Вычисляю квадрат 3... → 9

# Повторные вызовы — "hits" (берём из кеша)
print(square(2))  # (ничего не печатает) → 4
print(square(3))  # (ничего не печатает) → 9

# Новый вызов — ещё один "miss" (maxsize=3, но 2 и 3 уже в кеше)
print(square(4))  # Вычисляю квадрат 4... → 16

# Проверяем статистику
print(square.cache_info())
```

**Вывод:**
```
CacheInfo(hits=2, misses=3, maxsize=3, currsize=3)
```
- **`hits=2`** → два раза использовали кеш (для `square(2)` и `square(3)`).  
- **`misses=3`** → три раза вычисляли (`2`, `3`, `4`).  
- **`maxsize=3`** → кеш хранит максимум 3 значения.  
- **`currsize=3`** → сейчас в кеше лежат `2`, `3`, `4`.  

---

### 🧹 **Пример 2: Очистка кеша (`cache_clear`)**
Допустим, у нас есть функция, которая загружает данные пользователя:

```python
from functools import lru_cache


@lru_cache(maxsize=100)
def get_user_data(user_id):
    print(f"Загружаю данные пользователя {user_id}...")
    return f"Данные пользователя {user_id}"


# Первый вызов — данные загружаются
print(get_user_data(1))  # Загружаю данные пользователя 1... → Данные пользователя 1

# Второй вызов — данные берутся из кеша
print(get_user_data(1))  # (ничего не печатает) → Данные пользователя 1

# Очищаем кеш
get_user_data.cache_clear()

# Теперь снова будет загрузка
print(get_user_data(1))  # Загружаю данные пользователя 1... → Данные пользователя 1
```

**Что произошло?**  
1. Первый вызов → `miss` (данные загружены в кеш).  
2. Второй вызов → `hit` (данные взяты из кеша).  
3. `cache_clear()` → кеш очищен.  
4. Третий вызов → снова `miss` (кеш пуст, данные загружаются заново).  

---

### 📊 **Пример 3: Анализ кеша в рекурсивной функции (fib)**
Вернёмся к примеру с числами Фибоначчи:

```python
from functools import lru_cache


@lru_cache(maxsize=1000)
def fib(n):
    if n in (0, 1):
        return 1
    return fib(n - 1) + fib(n - 2)


# Первый вызов — долгий, т.к. вычисляет все fib от 0 до 35
print(fib(35))  # 14930352

# Второй вызов — мгновенный (все промежуточные fib уже в кеше)
print(fib(35))  # 14930352

# Смотрим статистику
print(fib.cache_info())
```

**Вывод:**
```
CacheInfo(hits=34, misses=36, maxsize=1000, currsize=36)
```
- **`hits=34`** → 34 раза использовали кеш (например, `fib(2)`, `fib(3)` и т. д.).  
- **`misses=36`** → 36 раз вычисляли (`fib(0)`, `fib(1)`, ..., `fib(35)`).  
- **`maxsize=1000`** → кеш может хранить до 1000 значений.  
- **`currsize=36`** → сейчас в кеше лежат `fib(0)` ... `fib(35)`.  

Если вызвать `fib.cache_clear()`, все эти значения удалятся, и следующий вызов `fib(35)` снова будет медленным.  

---

### 🎯 **Вывод: когда использовать эти методы?**
- **`.cache_info()`** → чтобы понять, насколько эффективно работает кеш.  
  - Если `hits` мало, возможно, `maxsize` слишком мал или функция редко вызывается с одинаковыми аргументами.  
- **`.cache_clear()`** → если данные изменились и старый кеш больше не актуален.  

Эти методы делают работу с `lru_cache` прозрачной и управляемой! 🚀

**Декораторы в Python** — это просто **"обёртки"**, которые добавляют дополнительное поведение к функциям или классам, не изменяя их исходный код.  

### 🔹 Простая аналогия:
Представь, что у тебя есть функция — это как **кофе**.  
Декоратор — это как **добавка к кофе** (сахар, молоко, сироп). Ты не меняешь сам кофе, но улучшаешь его вкус.  

### 🔹 Как это выглядит в коде?
Допустим, у тебя есть функция, которая говорит "Привет":  

```python
def say_hello():
    print("Привет!")
```

Но ты хочешь, чтобы перед приветствием выводилось `"Функция запущена"`, а после — `"Функция завершена"`.  

#### Вариант без декоратора (плохо, т.к. меняем исходную функцию):
```python
def say_hello():
    print("Функция запущена")  # Добавили вручную
    print("Привет!")
    print("Функция завершена")  # Добавили вручную
```

#### Вариант с декоратором (лучше!):
```python
# 1. Создаём декоратор (функцию-обёртку)
def simple_decorator(func):  # func — это функция, которую мы "улучшаем"
    def wrapper():
        print("Функция запущена")
        func()  # Вызываем исходную функцию
        print("Функция завершена")

    return wrapper


# 2. Применяем декоратор к функции
@simple_decorator
def say_hello():
    print("Привет!")


# 3. Вызываем функцию
say_hello()
```

**Вывод:**
```
Функция запущена
Привет!
Функция завершена
```

### 🔹 Что произошло?
1. Декоратор `@simple_decorator` **"обернул"** функцию `say_hello()` в функцию `wrapper()`.  
2. Теперь при вызове `say_hello()` выполняется не только `print("Привет!")`, но и код до и после.  

### 🔹 Зачем это нужно?
- Логирование (запись действий функции)  
- Замер времени выполнения  
- Проверка прав доступа  
- Кэширование результатов  
- И многое другое!  

### 🔹 Важно:
- Декораторы **не изменяют** исходную функцию, а **добавляют** к ней поведение.  
- Можно применять **несколько декораторов** к одной функции.  
- Декораторы работают не только с функциями, но и с классами.  

Если хочешь, разберём пример с замером времени работы функции! 🚀

**Зачем декораторы используют вложенные функции** и почему без них не обойтись.  

### 🔹 **Проблема: Декоратор должен "запомнить" состояние**  
Допустим, мы хотим создать декоратор `@count`, который считает вызовы функции.  

#### ❌ **Попытка без вложенной функции (не сработает)**  
```python
total = 0  # Глобальная переменная — плохая практика!


def count(f):
    global total  # Используем глобальную переменную
    total += 1
    return f


@count
def greet():
    print("Привет!")


greet()  # total = 1
greet()  # total = 2
```
**Минусы:**  
- `total` — глобальная переменная, её могут изменить другие функции.  
- Мы не можем привязать счётчик **к конкретной функции** (если декорируем две функции, счётчик будет общий).  

---

### ✅ **Решение: Вложенные функции**  
```python
def count(f):
    total = 0  # Локальная переменная для каждого декоратора

    def decorated(*args, **kwargs):
        nonlocal total  # Берём total из внешней функции
        total += 1
        print(f"Функция вызвана {total} раз")
        return f(*args, **kwargs)

    return decorated  # Возвращаем новую функцию
```
**Почему это работает?**  

1. **`total` живёт внутри `count()`**  
   - При каждом применении `@count` к функции создаётся **новая область видимости** для `total`.  
   - Если задекорировать две функции, у каждой будет **свой счётчик**.  

2. **`decorated()` запоминает `total`**  
   - Благодаря **замыканию (closure)** внутренняя функция `decorated` сохраняет доступ к переменным внешней функции (`count`), даже после её завершения.  

3. **`nonlocal` позволяет изменять `total`**  
   - Без `nonlocal` Python считал бы `total` локальной переменной `decorated`, и мы не смогли бы её изменять.  

---

### 🔹 **Что дают вложенные функции?**  
1. **Изоляция состояния**  
   - Каждый декоратор хранит свои переменные (`total`), не мешая другим.  

2. **Гибкость**  
   - Можно передавать аргументы в декоратор (если сделать ещё один уровень вложенности).  

3. **Чистый код**  
   - Не нужны глобальные переменные, которые усложняют отладку.  

---

### 🏁 **Пример с двумя функциями**  
```python
@count
def greet():
    print("Привет!")


@count
def bye():
    print("Пока!")


greet()  # Функция вызвана 1 раз
greet()  # Функция вызвана 2 раз
bye()  # Функция вызвана 1 раз (счётчик отдельный!)
```
**Вывод:**  
```
Функция вызвана 1 раз  
Привет!  
Функция вызвана 2 раз  
Привет!  
Функция вызвана 1 раз  
Пока!  
```

---

### 💡 **Итог: Зачем вложенность?**  
- **Чтобы сохранять состояние** (например, счётчик) между вызовами функции.  
- **Чтобы избежать глобальных переменных** (которые приводят к багам).  
- **Чтобы декоратор мог работать с любыми функциями** (благодаря `*args, **kwargs`).  

Если бы декоратор возвращал просто число или вызывал функцию сразу, он не мог бы **"обернуть"** её поведение. Вложенные функции — это способ **отложить выполнение** и добавить логику до/после вызова.  

Хочешь разберём декоратор с аргументами (например, `@repeat(3)` для повтора функции)? 😊

В декораторах всегда во вложенной функции пишем nonlocal к переменным из внешней функции, чтобы переменная сохраняла своё значение между вызовами.

### Генераторы

Такой объект не содержит все значения сразу — он выдаёт их по одному при каждой итерации. Чтобы получить значение, вызывается метод __next__() (обычно неявно — например, в цикле for).

```python
squares = (i**2 for i in range(10))
print(squares)
print(next(squares))
print(next(squares))
print(filter(lambda x: x % 2 == 0, squares))
print(list(filter(lambda x: x % 2 == 0, squares)))
```

```python
# Рассмотрим генераторную версию функции Фибоначчи:


def fib(n):  # это генератор-функция
    n_1, n_2 = 1, 1
    for i in range(n):
        yield n_1  # <- Точка остановки возвращает значение и приостанавливает функцию (отличие от return)
        n_1, n_2 = n_2, n_1 + n_2  # # <- Продолжение отсюда при следующем вызове


print(", ".join(str(x) for x in fib(10)))

# Вывод программы:
# 1, 1, 2, 3, 5, 8, 13, 21, 34, 55
```

Обратите внимание: внутри функции fib() есть цикл на n итераций, но выполняются они не все сразу. Генератор работает лениво — значения вычисляются и выдаются только по мере запроса. Если вы в цикле прерываете выполнение, недостающие значения так и не будут вычислены.
Это особенно важно при работе с большими или бесконечными последовательностями — генераторы экономят память и ускоряют работу.

### 🔹 **`fib(n)` — это функция-генератор**  
Да, **`def fib(n)` создаёт генератор**, несмотря на то что она принимает аргумент `n`.  

#### Почему?  
1. **Ключевое слово `yield`** — если функция содержит `yield`, она автоматически становится **генераторной функцией** (generator function).  
2. **При вызове `fib(n)` она возвращает генератор**, а не обычное значение.  
3. **Генератор выдаёт значения по одному** при каждом вызове `next()` или в цикле.  

---

### 🔄 **Как это работает?**  

#### 1️⃣ **Пример вызова**  
```python
gen = fib(3)  # Создаётся генератор (функция НЕ выполняется сразу)
print(next(gen))  # 1 (первое число)
print(next(gen))  # 1 (второе число)
print(next(gen))  # 2 (третье число)
print(next(gen))  # Ошибка StopIteration (закончились числа)
```

#### 2️⃣ **Что происходит в памяти?**  
| Шаг | Действие                                     | Состояние (`n_1`, `n_2`) |  
|-----|----------------------------------------------|--------------------------|  
| 1   | `gen = fib(3)` → создаётся генератор         | `n_1=1`, `n_2=1` (начальные значения) |  
| 2   | `next(gen)` → выполняется до `yield n_1`     | Возвращает `1`           |  
| 3   | `next(gen)` → продолжает с `n_1, n_2 = ...`  | `n_1=1`, `n_2=2`         |  
| 4   | `next(gen)` → снова `yield n_1`              | Возвращает `1`           |  
| 5   | `next(gen)` → обновляет `n_1, n_2`           | `n_1=2`, `n_2=3`         |  
| 6   | `next(gen)` → последний `yield n_1`          | Возвращает `2`           |  
| 7   | `next(gen)` → цикл завершён, `StopIteration` | —                        |  

---

### 🔹 **Чем это отличается от обычной функции?**  

| Обычная функция (`return`)               | Генератор (`yield`)                          |  
|------------------------------------------|----------------------------------------------|  
| Выполняется сразу до конца               | Выполняется **по частям** при каждом `next()` |  
| Возвращает **все значения сразу**        | Возвращает **по одному значению**            |  
| Переменные **удаляются** после `return`  | Переменные **сохраняются** между вызовами    |  
| Подходит для **готовых данных**          | Подходит для **больших/бесконечных** данных  |  

---

### 💡 **Почему `fib(n)` — это генератор, а не просто функция?**  
- Она **не вычисляет все числа сразу**, а выдаёт их **лениво** (lazy evaluation).  
- Можно использовать даже для **бесконечной последовательности** (если убрать `range(n)`).  
- Экономит память: не хранит весь список чисел, а **генерирует их на лету**.  

Пример бесконечного генератора Фибоначчи:  
```python
def infinite_fib():
    n_1, n_2 = 1, 1
    while True:  # Бесконечный цикл!
        yield n_1
        n_1, n_2 = n_2, n_1 + n_2


gen = infinite_fib()
print(next(gen))  # 1
print(next(gen))  # 1
print(next(gen))  # 2
# Можно вызывать бесконечно...
```

---

### 🏁 **Итог**  
- **`fib(n)` — это генератор**, потому что содержит `yield`.  
- Она **не выполняется сразу**, а возвращает специальный объект-генератор.  
- При каждом `next()` она **продолжает с места последнего `yield`**.  
- Это мощный инструмент для работы с **потоками данных**, которые не помещаются в память.  

Если хочешь, можем разобрать ещё примеры: например, как передавать значения в генератор через `send()` или как использовать генераторы в `asyncio`! 😊

```python
# новые аргументы в функции из-за декоратора, хоть в изначальной их не было


# ввод
@result_accumulator
def a_plus_b(a, b):
    return a + b


print(a_plus_b(3, 5, method="accumulate"))


# декоратор
# изначально func принимает только *args
def result_accumulator(func):
    result = []

    def wrap(
        *args, method="accumulate"
    ):  # args из func, добавляеся им.арг-т (обрабатывается обёрткой)
        result.append(func(*args))
        if method == "drop":
            temp = result.copy()
            result.clear()
            return temp

    return wrap
```

### 🔹 **Что происходит в вашем примере?**
1. **Исходная функция** `a_plus_b(a, b)` принимает только два позиционных аргумента.
2. **Декоратор** `result_accumulator` добавляет обёртку `wrap(*args, method="accumulate")`, которая:
   - Принимает любые позиционные аргументы (`*args`),
   - Добавляет **необязательный именованный аргумент** `method` со значением по умолчанию `"accumulate"`.

Теперь при вызове `a_plus_b(3, 5, method="accumulate")`:
- Аргументы `3` и `5` попадают в `*args`,
- `method="accumulate"` обрабатывается обёрткой.

```python
# бесконечный цикл
def cycle(spisok):
    while True:
        for item in spisok:
            yield item
```

In [ ]:
# 1

# fmt: off
from collections.abc import Generator, Sequence
from typing import Callable, Union

# fmt: on


def recursive_sum(*args: int) -> int:
    """Recursively sums a variable number of integer arguments."""
    if not args:
        return 0
    return args[-1] + recursive_sum(*args[:-1])

In [ ]:
# 2


def recursive_digit_sum(nmb: int) -> int:
    """Recursively sums the digits of a given integer."""
    if nmb // 10 > 0:
        return nmb % 10 + recursive_digit_sum(nmb // 10)
    return nmb % 10

In [ ]:
# 3


def make_equation(*coeffs: int) -> str:
    """Recursively creates a string representation of a polynomial equation."""
    if len(coeffs) == 1:
        return str(coeffs[0])
    line = ") * x " + ("- " if coeffs[-1] < 0 else "+ ") + str(coeffs[-1])
    return "(" + make_equation(*coeffs[:-1]) + line

In [ ]:
# 4


def answer(
    func: Callable[[int | str, int | str], int | str],
) -> Callable[[int | str, int | str], int | str]:
    """Decorate that wraps function result in a formatted string."""

    def decorated(*args: int | str, **kwargs: int | str) -> str:
        return f"Результат функции: {func(*args, **kwargs)}"

    return decorated

#### 5

```python
# изначально func принимает только *args
def result_accumulator(func):
    result = []

    def wrap(
        *args, method="accumulate"
    ):  # args из func, добавляеся им.арг-т (обрабатывается обёрткой)
        result.append(func(*args))
        if method == "drop":
            temp = result.copy()
            result.clear()
            return temp

    return wrap
```

In [ ]:
# 6


def merge(left: list[int], right: list[int]) -> list[int]:
    """Merge two sorted lists into a single sorted list."""
    result: list[int] = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left[0])
            left = left[1:]
        else:
            result.append(right[0])
            right = right[1:]

    if len(left) > 0:
        result += left
    if len(right) > 0:
        result += right
    return result


def merge_sort(arr: list[int]) -> list[int]:
    """Sorts a list using merge sort algorithm."""
    if len(arr) <= 1:
        return arr
    middle: int = int(len(arr) / 2)
    left: list[int] = merge_sort(arr[:middle])
    right: list[int] = merge_sort(arr[middle:])
    return merge(left, right)

In [ ]:
# 7


OutputType = Union[int, str, bool]


def same_type(
    func: Callable[[int | str], OutputType],
) -> Callable[[int | str], OutputType]:
    """Ensure all arguments passed to the function are of the same type."""

    def decorated(*args: int | str) -> OutputType:
        if len(set(map(type, args))) != 1:
            print("Обнаружены различные типы данных")
            return False
        return func(*args)

    return decorated

[<class 'int'>, <class 'str'>]


In [ ]:
# 8


def fibonacci(value: int) -> Generator[int]:
    """Decorate that yields the first n numbers in the Fibonacci sequence."""
    n_1, n_2 = 0, 1
    for _ in range(value):
        yield n_1
        n_1, n_2 = n_2, n_1 + n_2

In [ ]:
# 9


def cycle(arr: list[int]) -> Generator[list[int]]:
    """Decorate that infinitely yields elements from the list."""
    while arr:
        yield arr

In [ ]:
# 10


def make_linear(arr: Sequence[Union[int, Sequence[int]]]) -> list[int]:
    """Flattens a nested list structure into a single linear list."""
    result: list[int] = []

    for el in arr:
        if isinstance(el, Sequence):
            result.extend(make_linear(el))
        else:
            result.append(el)

    return result